In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
model_name = 'K40'
file_name = 'K40'

In [3]:
DATA_DIR = os.path.abspath("../../../prediction_model/data/raw_data")
SAVE_DIR = os.path.abspath("../../../prediction_model/data/")

## Dense Layer

In [4]:
dfDense = pd.read_pickle(os.path.join(DATA_DIR,'%s/1/benchmark_dense__20180924.pkl' %file_name))

for i in range(3,4):
    dfDense = pd.concat([dfDense,pd.read_pickle(os.path.join(DATA_DIR,'%s/%d/benchmark_dense__20181005.pkl' %(file_name, i)))])

In [5]:
ops = (dfDense['batchsize']
       * dfDense['dim_input'] 
       * dfDense['dim_output'])

memory_weights = dfDense['dim_input'] * dfDense['dim_output']
memory_in = dfDense['batchsize'] * dfDense['dim_input']
memory_out = dfDense['batchsize'] * dfDense['dim_output']


dfDense['optimizer'] = dfDense['optimizer'].replace({0:'opt_None',
                                                     1:'opt_SGD',
                                                     2:'opt_Adadelta',
                                                     3:'opt_Adagrad',
                                                     4:'opt_Momentum',
                                                     5:'opt_Adam',
                                                     6:'opt_RMSProp'})

dfDense['activation_fct'] = dfDense['activation_fct'].replace({0:'act_None',
                                                               1:'act_relu',
                                                               2:'act_tanh',
                                                               3:'act_sigmoid'})

dfDense['ops'] = ops
dfDense['memory_weights'] = memory_weights
dfDense['memory_in'] = memory_in
dfDense['memory_out'] = memory_out

In [6]:
one_hot_optimizer = pd.get_dummies(dfDense['optimizer'])

dfDense = dfDense.drop(labels='optimizer',axis=1)
dfDense = pd.concat([dfDense,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfDense['activation_fct'])

dfDense = dfDense.drop(labels='activation_fct',axis=1)
dfDense = pd.concat([dfDense,one_hot_activation],axis=1)

In [7]:
dfDense.describe()

,batchsize,dim_input,dim_output,gpu,precision,timeUsed_max,timeUsed_median,timeUsed_min,timeUsed_std,ops,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,25000.000000,25000.000000,25000.000000,25000.0,25000.0,25000.000000,25000.000000,25000.000000,25000.000000,2.500000e+04,...,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.00000,25000.000000,25000.000000,25000.000000,25000.000000
mean,21.526280,2047.171880,2040.910280,0.0,32.0,1.548109,1.507447,1.443333,0.039195,8.923048e+07,...,0.085600,0.083280,0.084280,0.498960,0.083920,0.08160,0.252760,0.246520,0.252400,0.248320
std,12.268515,1182.857265,1182.594204,0.0,0.0,1.473130,1.474100,1.463846,0.023793,1.035539e+08,...,0.279778,0.276311,0.277813,0.500009,0.277273,0.27376,0.434603,0.430993,0.434398,0.432047
min,1.000000,1.000000,1.000000,0.0,32.0,0.151882,0.132518,0.114017,0.000984,3.960000e+02,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,11.000000,1027.000000,1024.000000,0.0,32.0,0.542617,0.499624,0.444045,0.026451,1.503522e+07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,21.000000,2046.000000,2020.000000,0.0,32.0,0.953200,0.921721,0.862751,0.036209,4.947181e+07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,32.000000,3069.250000,3081.000000,0.0,32.0,2.028109,1.997172,1.918628,0.046802,1.266772e+08,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,1.000000,0.000000,1.000000,0.000000
max,47.000000,4095.000000,4095.000000,0.0,32.0,10.907278,10.866017,10.832601,0.529494,7.257065e+08,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000


In [8]:
dfDense.to_pickle(os.path.join(SAVE_DIR,'Data_dense_%s.pkl'%model_name))

## Convolutional Layers

In [9]:
dfConv = pd.read_pickle(os.path.join(DATA_DIR,'%s/0/benchmark_convolution__20180917.pkl' %file_name))

header = dfConv.columns

for i in range(2,3):
    dfConv = pd.concat([dfConv,pd.read_pickle(os.path.join(DATA_DIR,'%s/%i/benchmark_convolution__20181008.pkl' %(file_name, i)))])

In [10]:
padding_reduction = ((dfConv['padding']==0)
                     *(dfConv['kernelsize']-1))

elements_output = ((dfConv['matsize'] - padding_reduction)
                   / dfConv['strides'])**2

ops = (dfConv['batchsize']
       * elements_output
       * dfConv['kernelsize']**2
       * dfConv['channels_in']
       * dfConv['channels_out'])

memory_weights = (dfConv['kernelsize']**2
                  * dfConv['channels_in']
                  * dfConv['channels_out']
                  + dfConv['use_bias'] * dfConv['channels_out'])

memory_in = (dfConv['batchsize']
             * dfConv['matsize']**2
             * dfConv['channels_in'])

memory_out = (dfConv['batchsize']
              * elements_output
              * dfConv['channels_out'])


dfConv['elements_matrix'] = dfConv['matsize']**2
dfConv['elements_kernel'] = dfConv['kernelsize']**2

    
dfConv['ops'] = ops
dfConv['memory_weights'] = memory_weights
dfConv['memory_in'] = memory_in
dfConv['memory_out'] = memory_out


dfConv['optimizer'] = dfConv['optimizer'].replace({0:'opt_None',
                                                   1:'opt_SGD',
                                                   2:'opt_Adadelta',
                                                   3:'opt_Adagrad',
                                                   4:'opt_Momentum',
                                                   5:'opt_Adam',
                                                   6:'opt_RMSProp'})

dfConv['activation_fct'] = dfConv['activation_fct'].replace({0:'act_None',
                                                             1:'act_relu',
                                                             2:'act_tanh',
                                                             3:'act_sigmoid'})

dfConv['use_bias'] = np.uint8(dfConv['use_bias'])


dfConv.dropna(inplace=True)

In [11]:
one_hot_optimizer = pd.get_dummies(dfConv['optimizer'])

dfConv = dfConv.drop(labels='optimizer',axis=1)
dfConv = pd.concat([dfConv,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfConv['activation_fct'])

dfConv = dfConv.drop(labels='activation_fct',axis=1)
dfConv = pd.concat([dfConv,one_hot_activation],axis=1)

In [12]:
dfConv.describe()

,batchsize,channels_in,channels_out,gpu,kernelsize,matsize,padding,precision,strides,timeUsed_max,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,25000.000000,25000.000000,25000.000000,25000.0,25000.000000,25000.000000,25000.000000,25000.0,25000.000000,25000.000000,...,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.00000,25000.000000
mean,21.664800,67.081240,66.379040,0.0,4.002200,256.223840,0.496920,32.0,2.500800,34.251666,...,0.085000,0.082120,0.082880,0.498360,0.088520,0.081960,0.250920,0.250960,0.25172,0.246400
std,12.241619,341.151859,322.745822,0.0,2.004284,147.433493,0.500001,0.0,1.118557,60.882647,...,0.278887,0.274553,0.275706,0.500007,0.284055,0.274309,0.433551,0.433574,0.43401,0.430923
min,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.000000,32.0,1.000000,0.260119,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,11.000000,10.000000,10.000000,0.0,2.000000,128.000000,0.000000,32.0,2.000000,4.302280,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,22.000000,19.000000,19.000000,0.0,4.000000,256.000000,0.000000,32.0,2.000000,12.697799,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,32.000000,39.000000,38.000000,0.0,6.000000,385.000000,1.000000,32.0,4.000000,36.886129,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.00000,0.000000
max,47.000000,9978.000000,9899.000000,0.0,7.000000,512.000000,1.000000,32.0,4.000000,937.858963,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [13]:
dfConv.to_pickle(os.path.join(SAVE_DIR,'Data_convolution_%s.pkl'%model_name))